In [ ]:
import numpy as np
import pandas as pd
from gurobipy import *

In [ ]:
!pip install gurobipy  # install gurobipy, if not already installed
import gurobipy as gp

In [ ]:
df = pd.read_excel('real_distances_10customers.xls', index_col=0) # dataframe
df

,0,1,2,3,4,5,6,7,8,9,10
0,0.0,225.4,351.3,324.2,204.8,272.1,349.7,340.2,386.0,141.8,269.8
1,228.1,0.0,150.1,123.0,62.8,113.6,209.4,190.9,226.7,103.6,73.6
2,303.5,101.6,0.0,145.5,124.1,153.2,263.8,245.3,244.6,179.0,86.1
3,320.5,123.2,176.9,0.0,141.1,115.8,167.9,149.4,127.6,196.0,63.7
4,211.2,60.4,174.2,145.4,0.0,71.9,160.7,142.2,188.0,86.7,92.8
5,273.3,107.8,212.4,113.9,71.7,0.0,110.6,92.1,137.9,148.8,88.4
6,355.4,195.7,306.5,161.0,149.5,94.1,0.0,39.1,102.8,229.6,182.5
7,328.3,162.8,267.4,121.9,126.7,55.0,52.8,0.0,80.1,203.8,143.4
8,409.0,242.7,303.0,143.5,203.2,143.7,129.3,116.5,0.0,284.5,195.7
9,142.3,104.0,229.9,202.8,83.4,150.7,222.8,215.5,261.3,0.0,148.4


In [ ]:
nodes = range(df.shape[0])# List of nodes

In [ ]:
nodes

range(0, 11)

In [ ]:
dist = [[df[j][i] for j in nodes] for i in nodes]# Build distance matrix

In [ ]:
dist

[[0.0, 225.4, 351.3, 324.2, 204.8, 272.1, 349.7, 340.2, 386.0, 141.8, 269.8],
 [228.1, 0.0, 150.1, 123.0, 62.8, 113.6, 209.4, 190.9, 226.7, 103.6, 73.6],
 [303.5, 101.6, 0.0, 145.5, 124.1, 153.2, 263.8, 245.3, 244.6, 179.0, 86.1],
 [320.5, 123.2, 176.9, 0.0, 141.1, 115.8, 167.9, 149.4, 127.6, 196.0, 63.7],
 [211.2, 60.4, 174.2, 145.4, 0.0, 71.9, 160.7, 142.2, 188.0, 86.7, 92.8],
 [273.3, 107.8, 212.4, 113.9, 71.7, 0.0, 110.6, 92.1, 137.9, 148.8, 88.4],
 [355.4, 195.7, 306.5, 161.0, 149.5, 94.1, 0.0, 39.1, 102.8, 229.6, 182.5],
 [328.3, 162.8, 267.4, 121.9, 126.7, 55.0, 52.8, 0.0, 80.1, 203.8, 143.4],
 [409.0, 242.7, 303.0, 143.5, 203.2, 143.7, 129.3, 116.5, 0.0, 284.5, 195.7],
 [142.3, 104.0, 229.9, 202.8, 83.4, 150.7, 222.8, 215.5, 261.3, 0.0, 148.4],
 [270.3, 77.7, 127.5, 61.6, 90.9, 85.2, 195.8, 177.3, 172.5, 145.8, 0.0]]

In [ ]:
demand = np.ones(len(nodes))

In [ ]:
demand

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
veh_cap = 5 # vehicle capacity
k = 2 # number of vehicles

In [ ]:
#Create a new model
vrp = Model("vrp_model")

Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
#Create variables
x = vrp.addVars(nodes, nodes, lb = 0,vtype = GRB.BINARY,name = 'x')
u = vrp.addVars(nodes, lb = 0,vtype = GRB.INTEGER,name = 'u')

In [ ]:
#Add constraint
vrp.addConstrs(((quicksum(x[i,j] for j in nodes if j != i) == 1) for i in nodes if i != 0), name = '1');
vrp.addConstrs(((quicksum(x[i,j] for i in nodes if i != j) == 1) for j in nodes if j != 0), name = '2');
vrp.addConstr(((quicksum(x[0,j] for j in nodes) == k)), name = '3');
vrp.addConstr(((quicksum(x[i,0] for i in nodes) == k)), name = '4');
vrp.addConstrs(((u[i] - u[j] + veh_cap*x[i,j] <= veh_cap - demand[j]) for i in nodes for j in nodes if i!=j and i != 0 and j != 0), name = '5');
vrp.addConstrs(((u[i] <= veh_cap) for i in nodes), name = '6');
vrp.addConstrs(((u[i] >= demand[i]) for i in nodes), name = '7');

In [ ]:
#set objective function
vrp.setObjective((quicksum(dist[i][j]*x[i,j] for i in nodes for j in nodes if i != j)),GRB.MINIMIZE)

In [ ]:
vrp.setParam("TimeLimit", 600)   # time limit
#vrp.setParam("OutputFlag", 0)     # do not print on console

Set parameter TimeLimit to value 600


In [ ]:
vrp.update()
vrp.optimize()


Cutting planes:
  Learned: 1
  Gomory: 1
  MIR: 2
  Flow cover: 4
  Zero half: 9

Explored 133702 nodes (989472 simplex iterations) in 51.64 seconds (15.71 work units)
Thread count was 2 (of 2 available processors)

Solution count 10: 1721.1 1723.7 1728.1 ... 2052

Optimal solution found (tolerance 1.00e-04)
Best objective 1.721100000000e+03, best bound 1.721000000000e+03, gap 0.0058%


In [ ]:
status = vrp.status

object_Value = vrp.objVal

print()
print("model status is: ", status)
print()
print("Objective value is: ", object_Value)


model status is:  2

Objective value is:  1721.1


In [ ]:
# Print decision varibales which are not zeros
if status !=3 and status != 4:
    for v in vrp.getVars():
        if vrp.objVal < 1e+99  and v.x!=0:
            print('%s %f'%(v.Varname,v.x))

x[0,3] 1.000000
x[0,4] 1.000000
x[1,9] 1.000000
x[2,1] 1.000000
x[3,8] 1.000000
x[4,10] 1.000000
x[5,0] 1.000000
x[6,7] 1.000000
x[7,5] 1.000000
x[8,6] 1.000000
x[9,0] 1.000000
x[10,2] 1.000000
u[0] 1.000000
u[1] 4.000000
u[2] 3.000000
u[3] 1.000000
u[4] 1.000000
u[5] 5.000000
u[6] 3.000000
u[7] 4.000000
u[8] 2.000000
u[9] 5.000000
u[10] 2.000000


In [ ]:
# printing routes
if status != 3 and status != 4:
    vis = []
    Sol_x = np.zeros([len(nodes), len(nodes)])
    for i in nodes:
        for j in nodes:
            if vrp.objVal < 1e+99:
                Sol_x[i,j] = x[i,j].getAttr("X")
            else:
                error_status = True
                ofvv = 1e+99
            if 1 - 0.00001 <= Sol_x[i,j] <= 1 + 0.00001:
                vis.append((i,j))

visited = np.array(vis)
solution = []
if visited[0][0] == 0:
    sol = [visited[0][0], visited[0][1]]
elif visited[0][0] !=0 and visited[0][1] == 0:
    sol = [visited[0][1], visited[0][0]]
else:
    print('First tuple should include depot 0')
visited = np.delete(visited,0, axis = 0)

solution = []
for i in visited:
    try:
        next_ind = int(np.where(visited[:,0] == sol[-1])[0])
        sol.append(visited[next_ind][1])
        visited = np.delete(visited,next_ind, axis = 0)
    except:
        next_ind = int(np.where(visited[:,1] == sol[-1])[0])
        sol.append(visited[next_ind][0])
        visited = np.delete(visited,next_ind, axis = 0)

    if sol[0] == sol[-1]:
        sol = np.asarray(sol)
        solution.append(sol)
        used = []
        for j in solution:
            for k in j:
                used.append(k)
        remain = list(set(nodes) - set(used))
        if remain == []:
            break
        sol=[visited[0][0], visited[0][1]]
        visited = np.delete(visited,0, axis = 0)

print('optimal Solution is:', solution)

optimal Solution is: [array([0, 3, 8, 6, 7, 5, 0]), array([ 0,  4, 10,  2,  1,  9,  0])]


<ipython-input-21-dc341a2ce799>:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  next_ind = int(np.where(visited[:,0] == sol[-1])[0])
